# Word Embeddings and Connections

## Learning Goals

Today our learning goals are to be able to...

1. Recognize and describe scenarios where word embeddings may be useful
2. Use pretrained word embeddings to solve problems
3. have intuitions about meaning encoded or not encoded within word vectors

## Today's Activity

Not only are word embeddings designed to be good encodings of word meaning, they are typically designed to encode *word similarity*. Luckily for us, identifying word similarity is a very common task in puzzle games, including the (hopefully somewhat recently) popular NYT game Connections. 

The task in this game is to partition 16 words into 4 groups of four which share some common property. This is typically a meaning-related property (though sometimes a particular instance of Connections involves spelling tricks, which our journey into tokenization tells us we may have trouble with). 

At the very least, this will be a reasonable test of the effectiveness of a vector embedding trained on co-occurance statistics!

To begin, we'll load in our word-vectors of choice: 50-dimensional [GloVe](https://nlp.stanford.edu/projects/glove/) vectors. Like, skip-gram/word2vec, GloVe is an approach to training word embeddings based on word co-occurance. The primary difference is that GloVe vectors trained not on context prediction but to directly estimate *global* co-occurance probabilities (i.e., the *Glo* in GloVe). GloVe models pretrained over 6 billions words with various embedding sizes are available freely, so let's load in the 50d embeddings from the provided file.

In [44]:
import numpy as np

embeddings = {}
with open("embeddings/glove.6B.50d.txt") as embeddings_f:
    for line in embeddings_f:
        word, *embed = line.split()
        embeddings[word] = np.array([float(x) for x in embed])

Now let's set up our game of Connections. Luckily, the game only really consists of 16 words, each represented as a string. Thus our game state can be entirely represented by a list with initial length 16!

Here's the daily game from 10/27/2024. 

In [45]:
# 10/27/24
game_1 = ["fresh", "prince", "bel", "air", 
          "quality", "bar", "cute", "mermaid", 
          "lux", "wise", "tramp", "mood", 
          "feeling", "smart", "mole", "rascals"]

We can also verify that all of the words are in our vocab. The following snipped would print any out-of-vocab words.

In [46]:
for w in game_1:
    if w not in embeddings:
        print(w)

Now onto strategy. 

We can compute the similarity between two word vectors by computing their *cosine similarity*. That is, we can compute
$$ \text{sim}(w, v) = \frac{w \cdot v}{\lvert w \rvert \lvert v \rvert} $$

Our goal will be to iteratively choose the combination of 4 words that maximizes the similarity between all of the words. 

We've defined the similarity between two words in terms of vectors, but it's still unclear what the similarity score between four words should be. For now, we'll define that as the sum of the similarities between every pair of words in the set of 4! **Write the two functions below to compute scores for both pairs of words and 4-tuples!**

In order to do that, a few functions might be handy to save you some time. First, the `itertools` library has a function called `combinations(x, k)` that gets all possible unique $k$-tuples in x (assuming order doesn't matter). `permutations` does the same, but considers order.

In [47]:
from itertools import combinations, permutations

x = ["A", "B", "C", "D"]
for y in combinations(x, 2):
    print(y)

print("---")
for y in permutations(x, 2):
    print(y)

('A', 'B')
('A', 'C')
('A', 'D')
('B', 'C')
('B', 'D')
('C', 'D')
---
('A', 'B')
('A', 'C')
('A', 'D')
('B', 'A')
('B', 'C')
('B', 'D')
('C', 'A')
('C', 'B')
('C', 'D')
('D', 'A')
('D', 'B')
('D', 'C')


`norm` from `numpy.linalg` also computes the norm (i.e., magnitude) of a vector for you. You can confirm it works in the cell below.

In [48]:
from numpy.linalg import norm

v = np.array([1, 3, 7, 2, -1])

# Use norm
print(norm(v))

# Compute norm "by hand"
print(np.sqrt(sum(x**2 for x in v)))

8.0
8.0


In [56]:
from typing import Mapping, Sequence

def score_pair(embeddings : Mapping[str, Sequence[float]], w_1 : str, w_2 : str) -> float:
    #TODO
    return 0.0

def score_row(embeddings : Mapping[str, Sequence[float]], row : Sequence[str]) -> float:
    # TODO
    return 0.0

Now let's build a model to make a guess. We want to 

1. Score all possible combinations of 4 words that have not been selected (i.e., are in the parameter `remaining_words`)
2. Find and return the top-`k` highest scoring combinations

When guessing, we can proceed down the list until we guess right, at which point we can remove the guessed words from `remaining_words` and make a second guess!

Write `make_guess`, which should accomplish this

In [ ]:
def make_guess(embeddings : Mapping[str, Sequence[float]], remaining_words : Sequence[str], k : int = 10) -> Sequence[:
    # TODO

Now use the following box to play the game! What kinds of semantic relationships does it get right? What does it get wrong? 

Now let's consider how well our model does with respect to the solution. Here are the true partitions, along with the NYT provided category labels. 

In [50]:
solution = [
    ["air", "feeling", "mood", "quality"],   # Ambience
    ["cute", "fresh", "smart", "wise"],      # Sassy
    ["bar", "bel", "lux", "mole"],           # Units 
    ["mermaid", "prince", "rascal", "tramp"] # The little _
]

For the purpose of evaluating the model, we can begin to ask how well the model can guess a correct partition. There are a couple of ways to measure this. For example, we can do what the NYT does: Count the number of guesses it would take to ge tthe correct answer. This is equivalent to getting the the minimum 0-indexed *rank* (i.e., position in the output list) of a correct grouping, removing those words and repeating. Our score would be the sum of all of those ranks! We win if the sum of ranks is less than the number of mistakes we're allowed to make (5 by the NYT's rules). 

Another approach is to take the *top-k* accuracy. That is, compute model accuracy, but the answer counts as correct if it's in the top-k provided options.

One other (more sensitive) measure is the MRR (Mean Reciprocal Rank). Here, for each prompt, we take the *reciprocal* of the best answer's rank (i.e., $\frac{1}{\text{rank}}$). The MRR is, as the name implies, the mean of all of these reciprocals. This is also the reciprocal of the harmonic mean of the ranks!

Of course, computing these over a single example is a bit lackluster. If you're not up to designing your own games to test, you can consider evaluating this model with [this Kaggle dataset](https://www.kaggle.com/datasets/eric27n/the-new-york-times-connections) of historical Connections games. The data is loaded in in the next cell.

In [51]:
import csv 
connections_rows = []
with open("data/connections.csv") as in_f:
    data_reader = csv.DictReader(in_f)

    for row in data_reader:
        connections_rows.append(row)

n_games = max([int(row["Game ID"]) for row in connections_rows])
n_games

537

In [52]:
# This code assumes rows are ordered so this is a bit less time consuming

connections_rows = sorted(connections_rows, key=lambda x: int(x["Game ID"]))

eval_data = []
current_game_id = 1
current_game_rows = []
for row in connections_rows:
    if int(row["Game ID"]) == current_game_id:
        current_game_rows.append(row)

    else:
        # Insert the found game
        board = [row["Word"].lower() for row in current_game_rows]
        solution = [[row["Word"].lower() for row in current_game_rows 
                     if (int(row["Group Level"]) == level)] 
                    for level in range(4)]
        eval_data.append((board, solution))

        # Reset for next iter
        current_game_rows = [row]
        current_game_id += 1

In [53]:
# Filter games that have out-of-vocab words
def in_vocab(embeddings : Mapping[str, Sequence[float]], board : Sequence[str]):
    for word in board:
        if word not in embeddings:
            return False
    return True

filtered_eval_data = [(board, solution) for board, solution in eval_data if in_vocab(embeddings, board)]

print("{} of {} in-vocab games to evaluate on".format(len(filtered_eval_data), len(eval_data)))

NameError: name 'Mapping' is not defined

In [54]:
def evaluate(embeddings : Mapping[str, Sequence[float]], data) -> float:
    # TODO Compute some metric to measure our model's performance. 
    return 0.0
    

NameError: name 'Mapping' is not defined

In [55]:
evaluate(embeddings, eval_data)

NameError: name 'evaluate' is not defined

If you have additional time and want to get even more familiar with word embeddings and their quirks, consider [Semantle](https://semantle.com/), a word-game that directly evaluates your ability to intuit meaning as represented by word vectors!